In [1]:
import cv2
import numpy as np
from keras.preprocessing import image as im
from keras import backend as k
import tensorflow as tf
from keras.models import load_model
model=load_model('finalbestmodel.hdf5')

C:\Users\Praneet\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def traverseCroppedSections():
    labelList = []
    for x, y, w, h in coordinates:
        croppedSection = cropImage(x, y, w, h, nonfilledimage)
        #label = tesseract.image_to_string(croppedSection)
        #labelList.append(label)
    # push these labels to database along with the coordinates and index

def cropImage(x, y, w, h, image):
    # Crop image here
    croppedImage = image[y:y + h,x:x + w]
    return croppedImage

# crop nonfilled
def getNonfilledCroppedSections(coordinates):
    for x, y, w, h in coordinates:
        nonfilledcroppedsection = cropImage(x, y, w, h, nonfilledimage)
        height = nonfilledcroppedsection.shape[0]
        width = nonfilledcroppedsection.shape[1]
        result = cv2.matchTemplate(filledimage, nonfilledcroppedsection, cv2.TM_CCOEFF)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        crop_x, crop_y, crop_w, crop_h = max_loc[0], max_loc[1], width, height
        filledcroppedsection = cropImage(crop_x, crop_y, crop_w, crop_h, filledimage)
        setDifference = np.subtract(filledcroppedsection, nonfilledcroppedsection)
        converted_to_text = ocr(setDifference)
        print(converted_to_text)
        
def ocr(image):
    kernel_line = np.ones((10, 80), np.uint8)
    dilated_line = cv2.dilate(image, kernel_line, iterations=1)
    
    im2, ctrs_line, hier = cv2.findContours(dilated_line.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    sorted_ctrs_line = sorted(ctrs_line, key=lambda ctr: cv2.boundingRect(ctr)[1])
    word_list = []
    
    for i, ctr_line in enumerate(sorted_ctrs_line):
        line_x, line_y, line_w, line_h = cv2.boundingRect(ctr_line)
        # if condition for too small contours
        if line_w * line_h > 115:
            line = image[line_y:line_y + line_h, line_x:line_x + line_w]
            cv2.imshow('Non Filled', line)
            cv2.waitKey(0)
            cv2.destroyAllWindows()        
            kernel_word = np.ones((10, 15), np.uint8)
            dilated_word = cv2.dilate(line, kernel_word, iterations=1)

            im2, ctrs_word, hier = cv2.findContours(dilated_word.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            sorted_ctrs_word = sorted(ctrs_word, key=lambda ctr: cv2.boundingRect(ctr)[1])

            for i, ctr_word in enumerate(sorted_ctrs_word):
                word_x, word_y, word_w, word_h = cv2.boundingRect(ctr_word)
                # if condition for too small contours
                if word_w * word_h > 115:
                    word = line[word_y:word_y + word_h, word_x:word_x + word_w]
                    cv2.imshow('Non Filled', word)
                    cv2.waitKey(0)
                    cv2.destroyAllWindows()            
                    kernel_char = np.ones((10, 1), np.uint8)
                    dilated_char = cv2.dilate(word, kernel_char, iterations=1)

                    im2, ctrs_char, hier = cv2.findContours(dilated_char.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    sorted_ctrs_char = sorted(ctrs_char, key=lambda ctr: cv2.boundingRect(ctr)[1])

                    character_list = []

                    for i, ctr_char in enumerate(sorted_ctrs_char):
                        char_x, char_y, char_w, char_h = cv2.boundingRect(ctr_char)
                        char = word[word_y:word_y + word_h, word_x:word_x + word_w]
                        #character = prediction(char)
                        #character_list.append(character)

                    #word_list.append("".join(character_list))
    #recognized_text = " ".join(word_list)
    return "recognized_text"
                
def prediction(char_image):
    squared = makeSquare(char_image)
    size28 = resize_to_pixel(28, squared)
    predict_img = im.img_to_array(size28)
    predict_img = np.expand_dims(predict_img, axis = 0)
    predictedarray = model.predict(predict_img)
    
    top_values, top_indices = k.get_session().run(tf.nn.top_k(model.predict_proba(predict_img), k=3))
    predicted_class = label_dictionary[top_indices[0]]
    return predicted_class

# match that nonfilled cropped template in filled ***************************************** done
# crop the matched template from filled ***************************************** done
# apply set difference between nonfilled template and filled template ***************************************** done
# check for contours in setdifferenced template ***************************************** done
## line detection ***************************************** done
## word detection ***************************************** done
## predict characters ***************************************** done
## return word
## return wordlist
# make key value pair of label and word list

def makeSquare(not_square):
    # Adds black pixels as padding
    
    BLACK = [0, 0, 0]
    img_dim = not_square.shape
    height = img_dim[0]
    width = img_dim[1]
    if (height == width):
        square = not_square
        return square
    else:
        doublesize = cv2.resize(not_square, (2 * width, 2 * height), interpolation = cv2.INTER_CUBIC)
        height = height * 2
        width = width * 2
        if (height > width):
            pad = (height - width)/2
            pad = int(pad)
            #doublesize = int(doublesize)
            doublesize_square = cv2.copyMakeBorder(doublesize, 0, 0, pad,\
                                                  pad, cv2.BORDER_CONSTANT, value = BLACK)
        else:
            pad = (width - height)/2
            pad = int(pad)
            doublesize_square = cv2.copyMakeBorder(doublesize, pad, pad, 0, 0,\
                                                  cv2.BORDER_CONSTANT, value = BLACK)
    doublesize_square_dim = doublesize_square.shape
    return doublesize_square

def resize_to_pixel(dimensions, image):
    
    buffer_pix = 4
    dimensions = dimensions - buffer_pix
    squared = image
    r = float(dimensions)/squared.shape[1]
    dim = (dimensions, int(squared.shape[0] * r))
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    img_dim2 = resized.shape
    height_r = img_dim2[0]
    widht_r = img_dim2[1]
    BLACK = [0, 0, 0]
    if (height_r > widht_r):
        resized = cv2.copyMakeBorder(resized, 0, 0, 0, 1, cv2.BORDER_CONSTANT, value = BLACK)
    if (height_r < widht_r):
        resized = cv2.copyMakeBorder(resized, 1, 0, 0, 0, cv2.BORDER_CONSTANT, value = BLACK)
    p = 2
    ReSizedImg = cv2.copyMakeBorder(resized, p, p, p, p, cv2.BORDER_CONSTANT, value = BLACK)
    img_dim = ReSizedImg.shape
    height = img_dim[0]
    width = img_dim[1]
    return ReSizedImg


In [3]:
nonfilledimage = cv2.imread('nonfilled.jpg',0)
#nonfilledimage = cv2.bitwise_not(nonfilledimage)
nonfilledimage = cv2.threshold(nonfilledimage, 100, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
cv2.imshow('Non Filled', nonfilledimage)
cv2.waitKey(0)
cv2.destroyAllWindows()

filledimage = cv2.imread('filled.jpg',0)
#filledimage = cv2.bitwise_not(filledimage)
filledimage = cv2.threshold(filledimage, 100, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
cv2.imshow('Filled', filledimage)
cv2.waitKey(0)
cv2.destroyAllWindows()

coordinates = [[101,464,1000,44],[90,621,448,44],[557,623,448,49],[92,541,1017,56],[94,1383,423,37],[438,1416,287,53]]

label_dictionary = {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: 'a',
                    11: 'b', 12: 'd', 13: 'e', 14: 'f', 15: 'g', 16: 'h', 17: 'i', 18: 'j', 19: 'l', 20: 'm',
                    21: 'n', 22: 'q', 23: 'r', 24: 't', 25: 'y', 26: 'A', 27: 'B', 28: 'C', 29: 'D', 30: 'E',
                    31: 'F', 32: 'G', 33: 'H', 34: 'I', 35: 'J', 36: 'K', 37: 'L', 38: 'M', 39: 'N', 40: 'O',
                    41: 'P', 42: 'Q', 43: 'R', 44: 'S', 45: 'T', 46: 'U', 47: 'V', 48: 'W', 49: 'X', 50: 'Y',
                    51: 'Z'}
getNonfilledCroppedSections(coordinates)

recognized_text
recognized_text
recognized_text
recognized_text
recognized_text
recognized_text


In [1]:
label_dictionary = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'd', 'e', 'f',
                    'g', 'h', 'i', 'j', 'l', 'm', 'n', 'q', 'r', 't', 'y', 'A', 'B', 'C', 'D',
                    'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q',
                    'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
labels = {}
i = 0
for label in label_dictionary:
    labels[i] = label
    i = i + 1
print(labels)

{0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: 'a', 11: 'b', 12: 'd', 13: 'e', 14: 'f', 15: 'g', 16: 'h', 17: 'i', 18: 'j', 19: 'l', 20: 'm', 21: 'n', 22: 'q', 23: 'r', 24: 't', 25: 'y', 26: 'A', 27: 'B', 28: 'C', 29: 'D', 30: 'E', 31: 'F', 32: 'G', 33: 'H', 34: 'I', 35: 'J', 36: 'K', 37: 'L', 38: 'M', 39: 'N', 40: 'O', 41: 'P', 42: 'Q', 43: 'R', 44: 'S', 45: 'T', 46: 'U', 47: 'V', 48: 'W', 49: 'X', 50: 'Y', 51: 'Z'}
